audio_16k contiene los audios en formato .wav

In [15]:
import zipfile
import os

zip_file_path = "vs_release_16k.zip"
destination_directory = "data/"
folders_to_extract = ["audio_16k/", "meta/"]

if not os.path.exists(destination_directory):
    with zipfile.ZipFile(zip_file_path, 'r') as zf:
        for file in zf.namelist():
            if any(file.startswith(folder) for folder in folders_to_extract):
                zf.extract(file, destination_directory)

m0188_0_cough.wav  
m -> male  
cough -> cough  
0 -> index of that cough (individuals may have more than 1 cough registered)